1) Problem: Task Scheduling Precedence
Description
You have N tasks (numbered 0 to N−1). There are M precedence constraints of the form (A,B) meaning “Task A must be done before Task B.” Determine if you can find an order that respects these constraints. If multiple valid orders exist, print any one of them. If no valid order exists, print -1.

In [ ]:
#include <bits/stdc++.h>
using namespace std;

int main(){
    ios::sync_with_stdio(false); cin.tie(nullptr);
    int N,M; cin >> N >> M;
    vector<vector<int>> adj(N);
    vector<int> indeg(N,0);
    for(int i=0;i<M;i++){
        int A,B; cin>>A>>B;
        adj[A].push_back(B);
        indeg[B]++;
    }

    queue<int> q;
    for(int i=0;i<N;i++){
        if(indeg[i]==0) q.push(i);
    }

    vector<int> result;
    while(!q.empty()){
        int u=q.front(); q.pop();
        result.push_back(u);
        for(auto &nx : adj[u]){
            if(--indeg[nx]==0) q.push(nx);
        }
    }

    if((int)result.size()<N){
        cout << -1 << "\n";
    } else {
        for(auto &x : result) cout << x << " ";
        cout<<"\n";
    }
    return 0;
}


2) Problem: Group Hierarchy
Description
There are 𝑁 people, each belonging to a hierarchy of subgroups. We have Mcomparisons(𝐴<𝐵) meaning perso A outranked by 𝐵. Some pairs are unknown. Print how many “fully determined” people exist—ones whose relative rank to all others is deducible from the given comparisons.

In [ ]:
#include <bits/stdc++.h>
using namespace std;

int main(){
    ios::sync_with_stdio(false); cin.tie(nullptr);
    int N,M; cin>>N>>M;
    vector<vector<bool>> canBeat(N,vector<bool>(N,false));
    for(int i=0;i<M;i++){
        int A,B; cin>>A>>B;
        canBeat[A][B]=true;
    }
    // Floyd-Warshall for transitive closure
    for(int k=0;k<N;k++){
        for(int i=0;i<N;i++){
            for(int j=0;j<N;j++){
                canBeat[i][j]= canBeat[i][j] || (canBeat[i][k] && canBeat[k][j]);
            }
        }
    }
    // For each person p, check if p's rank vs all others is known
    int countDet=0;
    for(int p=0;p<N;p++){
        bool known=true;
        for(int x=0;x<N;x++){
            if(x==p) continue;
            bool pSmaller = canBeat[p][x];
            bool xSmaller = canBeat[x][p];
            if(!pSmaller && !xSmaller){
                known=false; break;
            }
        }
        if(known) countDet++;
    }
    cout<<countDet<<"\n";
    return 0;
}


3) Problem: Module Dependency Checker
Description You have N software modules, with M directed edges “Module A depends on B.” If any cycle exists, print CYCLE. Otherwise, print a topological ordering.

In [ ]:
#include <bits/stdc++.h>
using namespace std;

bool dfsCycle(int u, vector<int>&visit, vector<vector<int>>& adj){
    visit[u]=1;
    for(auto &nxt: adj[u]){
        if(visit[nxt]==0){
            if(dfsCycle(nxt,visit,adj)) return true;
        } else if(visit[nxt]==1){
            return true;
        }
    }
    visit[u]=2; 
    return false;
}

vector<int> ans;
void dfsTopo(int u, vector<int>& visited, vector<vector<int>>& adj){
    visited[u]=1;
    for(auto &nx: adj[u]){
        if(!visited[nx]) dfsTopo(nx, visited, adj);
    }
    ans.push_back(u);
}

int main(){
    ios::sync_with_stdio(false);cin.tie(nullptr);
    int N,M;cin>>N>>M;
    vector<vector<int>> adj(N);
    for(int i=0;i<M;i++){
        int A,B;cin>>A>>B;
        adj[A].push_back(B);
    }
    // check cycle
    vector<int> visit(N,0);
    for(int i=0;i<N;i++){
        if(!visit[i]){
            if(dfsCycle(i,visit,adj)) {cout<<"CYCLE\n"; return 0;}
        }
    }
    // no cycle => topological sort
    ans.clear();
    vector<int> visited2(N,0);
    for(int i=0;i<N;i++){
        if(!visited2[i]) dfsTopo(i, visited2, adj);
    }
    reverse(ans.begin(),ans.end());
    for(auto &x : ans) cout<<x<<" ";
    cout<<"\n";
    return 0;
}


4) Problem: Single Entrance Maze
Description
You have a grid with one entrance cell. Some cells are walls (#), others are open (.). You can move in 4 directions. Print how many open cells are reachable from the entrance.

In [ ]:
#include <bits/stdc++.h>
using namespace std;

int main(){
    ios::sync_with_stdio(false); cin.tie(nullptr);
    int R,C; cin>>R>>C;
    int sr,sc; cin>>sr>>sc; 
    vector<string> grid(R);
    for(int i=0;i<R;i++){
        cin>>grid[i];
    }
    vector<vector<bool>> visited(R,vector<bool>(C,false));
    int cnt=0;
    queue<pair<int,int>>q;
    q.push({sr,sc});
    visited[sr][sc]=true;
    int dirs[4][2]={{1,0},{-1,0},{0,1},{0,-1}};
    while(!q.empty()){
        auto [r,c]=q.front();q.pop();
        cnt++;
        for(auto &d: dirs){
            int nr=r+d[0], nc=c+d[1];
            if(nr>=0 && nr<R && nc>=0 && nc<C){
                if(!visited[nr][nc] && grid[nr][nc]=='.'){
                    visited[nr][nc]=true;
                    q.push({nr,nc});
                }
            }
        }
    }
    cout<<cnt<<"\n";
    return 0;
}


5) Problem: Two-Layer DAG Counting
Description
You have two layers of nodes: Layer A has N nodes, Layer B has M nodes. There are edges from A_i to B_j only. Output how many pairs (𝑎,𝑏) are connected by any path. (Here, it’s direct edges only, so count edges.)

In [ ]:
#include <bits/stdc++.h>
using namespace std;
int main(){
    ios::sync_with_stdio(false);cin.tie(nullptr);
    int N,M,E;cin>>N>>M>>E;
    // A: 0..N-1, B: N..N+M-1
    // direct edges only
    int countPairs=0;
    for(int i=0;i<E;i++){
        int a,b;cin>>a>>b; 
        // a in [0..N-1], b in [N..N+M-1]
        // assume input is consistent
    }
    // Actually we must read them. Then count E is straightforward.
    // But let's do a direct read:
    cout << E << "\n";
    return 0;
}


6) Problem: Strongly Connected Subgraphs
Description
You have a directed graph with 𝑁 nodes, M edges. Find the number of strongly connected components. Print that count.

In [ ]:
#include <bits/stdc++.h>
using namespace std;

vector<int>order, comp; 
int idx=0;
vector<bool>used;
void dfs1(int u, vector<vector<int>>& adj){
    used[u]=true;
    for(auto &nx:adj[u]){
        if(!used[nx]) dfs1(nx,adj);
    }
    order.push_back(u);
}
void dfs2(int u, int c, vector<vector<int>>& adjr){
    comp[u]=c;
    for(auto &nx: adjr[u]){
        if(comp[nx]==-1) dfs2(nx,c,adjr);
    }
}

int main(){
    ios::sync_with_stdio(false); cin.tie(nullptr);
    int N,M; cin>>N>>M;
    vector<vector<int>> adj(N), adjr(N);
    for(int i=0;i<M;i++){
        int u,v;cin>>u>>v;
        adj[u].push_back(v);
        adjr[v].push_back(u);
    }
    used.assign(N,false);
    for(int i=0;i<N;i++){
        if(!used[i]) dfs1(i,adj);
    }
    comp.assign(N,-1);
    int cidx=0;
    for(int i=0;i<N;i++){
        int u=order[N-1-i];
        if(comp[u]==-1){
            dfs2(u,cidx++,adjr);
        }
    }
    cout<< cidx <<"\n";
}


7) Problem: Count of Cycles of Length K in Undirected Graph
Description
Given an undirected graph with N vertices, count how many simple cycles of length = K exist. Print the count. If none, print 0.

Solution Code (C++) (inefficient example, suitable for small N)

In [ ]:
#include <bits/stdc++.h>
using namespace std;
int main(){
    ios::sync_with_stdio(false);cin.tie(nullptr);
    int N,M,K; cin>>N>>M>>K;
    vector<vector<bool>> adj(N,vector<bool>(N,false));
    for(int i=0;i<M;i++){
        int u,v;cin>>u>>v;
        adj[u][v]=adj[v][u]=true;
    }
    long long ans=0;
    vector<int> path(K);
    // naive DFS approach
    function<void(int,int,int)> dfs=[&](int start,int idx,int cur){
        path[idx]=cur;
        if(idx==K-1){
            if(adj[cur][start]) ans++;
            return;
        }
        for(int nxt=0;nxt<N;nxt++){
            if(adj[cur][nxt]){
                bool used=false;
                for(int i2=0;i2<=idx;i2++){
                    if(path[i2]==nxt){used=true;break;}
                }
                if(!used) dfs(start,idx+1,nxt);
            }
        }
    };
    for(int i=0;i<N;i++){
        dfs(i,0,i);
    }
    // each cycle counted multiple times depending on start and direction
    // adjust if needed
    cout<<ans/(2*K)<<"\n"; // typical correction factor
    return 0;
}


8) Problem: Minimum Distance to “Exit” Node
Description
You have a directed graph of N nodes, and node E is the exit. For each node i, compute the fewest edges needed to reach E. If unreachable, print -1.

In [ ]:
#include <bits/stdc++.h>
using namespace std;

int main(){
    ios::sync_with_stdio(false);cin.tie(nullptr);
    int N,M,E; cin>>N>>M>>E;
    vector<vector<int>> rev(N); 
    for(int i=0;i<M;i++){
        int u,v; cin>>u>>v;
        // normal edge u->v
        // store reverse v->u
        rev[v].push_back(u);
    }
    vector<int> dist(N,-1);
    queue<int>q; 
    dist[E]=0; 
    q.push(E);
    while(!q.empty()){
        int u=q.front();q.pop();
        for(int &pr : rev[u]){
            if(dist[pr]==-1){
                dist[pr]=dist[u]+1;
                q.push(pr);
            }
        }
    }
    for(int i=0;i<N;i++){
        cout<<dist[i]<<" ";
    }
    cout<<"\n";
    return 0;
}


9) Problem: Merge Classes by Compatibility
Description
You have N classes, each with edges meaning “these two classes are merge-compatible.” Find how many connected components exist (undirected).

In [ ]:
#include <bits/stdc++.h>
using namespace std;

void dfs(int s, vector<bool>&visited, vector<vector<int>>&adj){
    stack<int> st; st.push(s);
    visited[s]=true;
    while(!st.empty()){
        int u=st.top(); st.pop();
        for(auto &nx : adj[u]){
            if(!visited[nx]){ visited[nx]=true; st.push(nx); }
        }
    }
}

int main(){
    ios::sync_with_stdio(false);cin.tie(nullptr);
    int N,M;cin>>N>>M;
    vector<vector<int>> adj(N);
    for(int i=0;i<M;i++){
        int a,b;cin>>a>>b;
        adj[a].push_back(b);
        adj[b].push_back(a);
    }
    vector<bool> visited(N,false);
    int compCount=0;
    for(int i=0;i<N;i++){
        if(!visited[i]){ compCount++; dfs(i,visited,adj); }
    }
    cout<<compCount<<"\n";
    return 0;
}


10) Problem: Network Status: Which Nodes Are “Hubs”?
Description
A directed graph with N nodes, M edges. A “hub” is a node from which every other node is reachable. Print how many hubs exist.

In [ ]:
#include <bits/stdc++.h>
using namespace std;

int main(){
    ios::sync_with_stdio(false);cin.tie(nullptr);
    int N,M;cin>>N>>M;
    vector<vector<int>> adj(N);
    for(int i=0;i<M;i++){
        int u,v;cin>>u>>v;
        adj[u].push_back(v);
    }
    int ans=0;
    for(int start=0;start<N;start++){
        vector<bool> visited(N,false);
        stack<int> st; st.push(start);
        visited[start]=true; 
        int count=1;
        while(!st.empty()){
            int u=st.top(); st.pop();
            for(auto &nx: adj[u]){
                if(!visited[nx]){
                    visited[nx]=true; st.push(nx); count++;
                }
            }
        }
        if(count==N) ans++;
    }
    cout<<ans<<"\n";
    return 0;
}
